In [1]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [2]:
#from layer import Layer
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of edges that connects to neurons in next layer
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [3]:
#from layer import Layer

# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
        

In [4]:
def relu(x):
	return np.maximum(0., x)
    
def relu_prime(x):
    return np.greater(x, 0).astype(np.float32)

In [5]:
import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [6]:
# example of a function for calculating softmax for a list of numbers
from numpy import exp
 
# calculate the softmax of a vector
def softmax(vector):
    e = exp(vector)
    return e / e.sum()

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

        
    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network 
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        '''
        Fit function does the training. 
        Training data is passed 1-by-1 through the network layers during forward propagation.
        Loss (error) is calculated for each input and back propagation is performed via partial 
        derivatives on each layer.
        '''
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(relu, relu_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.340104
epoch 2/1000   error=0.294640
epoch 3/1000   error=0.285368
epoch 4/1000   error=0.279387
epoch 5/1000   error=0.273362
epoch 6/1000   error=0.269450
epoch 7/1000   error=0.263726
epoch 8/1000   error=0.256056
epoch 9/1000   error=0.248170
epoch 10/1000   error=0.243833
epoch 11/1000   error=0.237176
epoch 12/1000   error=0.230107
epoch 13/1000   error=0.222227
epoch 14/1000   error=0.222178
epoch 15/1000   error=0.216629
epoch 16/1000   error=0.206662
epoch 17/1000   error=0.207854
epoch 18/1000   error=0.204536
epoch 19/1000   error=0.204413
epoch 20/1000   error=0.197494
epoch 21/1000   error=0.194376
epoch 22/1000   error=0.195728
epoch 23/1000   error=0.191207
epoch 24/1000   error=0.189759
epoch 25/1000   error=0.188381
epoch 26/1000   error=0.187216
epoch 27/1000   error=0.185332
epoch 28/1000   error=0.183916
epoch 29/1000   error=0.182345
epoch 30/1000   error=0.180689
epoch 31/1000   error=0.178980
epoch 32/1000   error=0.177472
epoch 33/1000   e

In [9]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(relu, relu_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.261088
epoch 2/35   error=0.100000
epoch 3/35   error=0.100000
epoch 4/35   error=0.100000
epoch 5/35   error=0.100000
epoch 6/35   error=0.100000
epoch 7/35   error=0.100000
epoch 8/35   error=0.100000
epoch 9/35   error=0.100000
epoch 10/35   error=0.100000
epoch 11/35   error=0.100000
epoch 12/35   error=0.100000
epoch 13/35   error=0.100000
epoch 14/35   error=0.100000
epoch 15/35   error=0.100000
epoch 16/35   error=0.100000
epoch 17/35   error=0.100000
epoch 18/35   error=0.100000
epoch 19/35   error=0.100000
epoch 20/35   error=0.100000
epoch 21/35   error=0.100000
epoch 22/35   error=0.100000
epoch 23/35   error=0.100000
epoch 24/35   error=0.100000
epoch 25/35   error=0.100000
epoch 26/35   error=0.100000
epoch 27/35   error=0.100000
epoch 28/35   error=0.100000
epoch 29/35   error=0.100000
epoch 30/35   error=0.100000
epoch 31/35   error=0.100000
epoch 32/35   error=0.100000
epoch 33/35   error=0.100000
epoch 34/35   error=0.100000
epoch 35/35   error=0.1

In [10]:
print(x_test[0:3])

[[[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]]


In [11]:
print("Oğuz Mete Kara","Neural Network From Scratch","From Github")
print("Have a nice day !")

Oğuz Mete Kara Neural Network From Scratch From Github
Have a nice day !
